In [63]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

train = pd.read_csv("train.csv")
train

,id,content,type
0,0,informed equifax numerous time account amount ...,credit_reporting
1,1,writing regard late payment credit report bank...,debt_collection
2,2,incorrectly report balance report asked experi...,credit_reporting
3,3,attempted investigate remove inaccurate inform...,credit_reporting
4,4,trying trade vehicle newer model told salesman...,credit_reporting
...,...,...,...
64966,32480,sent correct information credit agency update ...,credit_reporting
64967,32481,scheduled appointment keybank discus unemploym...,retail_banking
64968,32482,year wondering credit never consistent never m...,credit_reporting
64969,32483,last day working tomorrow full week work emplo...,credit_card


In [64]:
print(train.isnull().sum())

id             0
content        2
type       32486
dtype: int64


In [65]:
train.dropna(subset=['content', 'type'], inplace=True)
train

,id,content,type
0,0,informed equifax numerous time account amount ...,credit_reporting
1,1,writing regard late payment credit report bank...,debt_collection
2,2,incorrectly report balance report asked experi...,credit_reporting
3,3,attempted investigate remove inaccurate inform...,credit_reporting
4,4,trying trade vehicle newer model told salesman...,credit_reporting
...,...,...,...
64966,32480,sent correct information credit agency update ...,credit_reporting
64967,32481,scheduled appointment keybank discus unemploym...,retail_banking
64968,32482,year wondering credit never consistent never m...,credit_reporting
64969,32483,last day working tomorrow full week work emplo...,credit_card


In [77]:
trainingset, validationset = train_test_split(train, test_size=0.2)

In [78]:
validationset

,id,content,type
9036,9036,may concern advised refusal pay notice claim d...,debt_collection
29219,29219,information sent third party proof identity ad...,credit_reporting
28646,28646,credit card elan financial servi open name nev...,credit_card
6422,6422,account inaccurate credit bureau reporting dif...,credit_reporting
28105,28105,ga experian tx experian tx intent file lawsuit...,credit_reporting
...,...,...,...
30717,30717,notified mortgage service company transferred ...,credit_reporting
21351,21351,report show increase added report called th sa...,credit_reporting
11166,11166,around year noticed fraudulent account listed ...,debt_collection
17889,17889,called equifax phone redirected several time r...,credit_reporting


In [68]:
input_train = trainingset['content']
output_train = trainingset['type']

model = Pipeline(steps=[('cv',CountVectorizer()),('lr',LogisticRegression(max_iter = 100000))])
model.fit(input_train, output_train)

Pipeline(steps=[('cv', CountVectorizer()),
                ('lr', LogisticRegression(max_iter=100000))])

In [69]:
testfile = pd.read_csv("test.csv")

In [70]:
predictions_dict = dict()

for i in range(0, len(testfile['id'])):
    predictions_dict[i] = ''

predictions_dict

predictions = pd.DataFrame(predictions_dict.items(), columns=['id','type'])
predictions

,id,type
0,0,
1,1,
2,2,
3,3,
4,4,
...,...,...
12989,12989,
12990,12990,
12991,12991,
12992,12992,


In [71]:
for j in range(0, len(testfile['content'])):

    test = testfile['content'][j]
    if type(test) != str:
        continue

    else:

        prob_array = model.predict_proba([test])

        x = np.max(prob_array)
        problist = prob_array.tolist()

        for i in range(0, len(problist[0])):
            if x == problist[0][i]:
                index = i
                break
        
        predictions['type'][j] = model.classes_[index]
    

predictions

,id,type
0,0,credit_reporting
1,1,debt_collection
2,2,retail_banking
3,3,retail_banking
4,4,credit_reporting
...,...,...
12989,12989,credit_card
12990,12990,credit_reporting
12991,12991,credit_reporting
12992,12992,retail_banking


In [72]:
predictions.to_csv("Predictions.csv", index=False)

In [79]:
model.score(validationset['content'], validationset['type'])

0.9402801292904417